# HRBP chatbot

这里，我们根据afy 的员工手册来做一个问答（为了取代summer（不是 qaq

## 1. 导入文件&数据预处理

In [1]:
import os
import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
api_key = os.environ['ZZZ_API_KEY']
api_key


'sk-zk239341a5c7ec929a1b6211d4f08130eff691d93161dbbe'

In [2]:
#test our api 
client = OpenAI(
    base_url = 'https://api.zhizengzeng.com/v1',
    api_key = api_key,
)

#response = client.chat.completions.create(
    #messages = [
        #{"role":"user", "content":"hello"}],
   # model = "gpt-5-nano"
#)
#response.choices[0].message.content

In [3]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "employee_handbook.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()
print(len(docs))

14


In [4]:
print(docs[0].page_content)

员⼯⼿册
致新员⼯的⼀封信:
亲爱的同事，
欢迎加⼊AFY⼤家庭！
在此，⼈⼒资源部谨代表全体同仁，对您的加⼊表⽰最热烈的祝贺与诚挚的欢迎。AFY致⼒于为每⼀
位员⼯提供宽⼴的发展舞台和多元的成⻓机会。我们相信，您的才华与激情，必将在这⾥得到充分释
放与绽放。
请将完整的报到资料交予⼈⼒资源部同事，我们将在1个⼯作⽇内完成⼊职信息录⼊与初步审核。待
审核通过后，我们将与您签署相关⼊职⽂件。⼈⼒资源部将在1周内完成所有相关流程，并向您返还
正式⽂件。完成报到与合同签署后，⼈⼒资源部将安排岗前培训，帮助您快速融⼊团队，明确发展路
径。
再次感谢您选择AFY。愿您在这⾥开启精彩职业旅程，与我们携⼿共创美好未来！
⸺⼈⼒资源部敬上
⼀、假勤管理办法
1. ⽬的
明确假勤要求，规范员⼯的假勤管理，使公司的管理规范化、制度化。作为薪资发放的依据之⼀。
2. 适⽤范围
公司全体⼈员。
3. 职责
3.1⼈⼒资源部负责制定、修改、执⾏、解释、说明公司考勤制度，进⾏⽇常考勤管理和每⽉考勤数据
核查与统计。所有考勤记录由⼈⼒资源部统⼀保管。


In [5]:
raw_text = "\n".join([d.page_content for d in docs])
raw_text[:200]

'员⼯⼿册\x01\n致新员⼯的⼀封信:\x01\n亲爱的同事，\n欢迎加⼊\x01AFY\x01⼤家庭！\x01\n在此，⼈⼒资源部谨代表全体同仁，对您的加⼊表⽰最热烈的祝贺与诚挚的欢迎。AFY\x01致⼒于为每⼀\n位员⼯提供宽⼴的发展舞台和多元的成⻓机会。我们相信，您的才华与激情，必将在这⾥得到充分释\n放与绽放。\n请将完整的报到资料交予⼈⼒资源部同事，我们将在\x011\x01个⼯作⽇\x01内完成⼊职信息录⼊与初步审核。待\n审核通过后，我们将与您签署相关'

In [6]:
len(raw_text)

9098

In [7]:
#去除 \x01 加粗文字
import re

clean_text = raw_text.replace("\x01", " ")
print(clean_text[:200])
print(len(clean_text))

员⼯⼿册 
致新员⼯的⼀封信: 
亲爱的同事，
欢迎加⼊ AFY ⼤家庭！ 
在此，⼈⼒资源部谨代表全体同仁，对您的加⼊表⽰最热烈的祝贺与诚挚的欢迎。AFY 致⼒于为每⼀
位员⼯提供宽⼴的发展舞台和多元的成⻓机会。我们相信，您的才华与激情，必将在这⾥得到充分释
放与绽放。
请将完整的报到资料交予⼈⼒资源部同事，我们将在 1 个⼯作⽇ 内完成⼊职信息录⼊与初步审核。待
审核通过后，我们将与您签署相关
9098


## Chunking

我们先用 最简单的 chunking试试看，我们总共有 9098个 character，我们就 400 一个 chunk， 其中 100 overlap

In [8]:
def chunk_text(text, chunk_size = 400, over_lap = 100):
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start+chunk_size, n)
        chunk = text[start:end].strip()

        if chunk:
            
            chunks.append(chunk)
        if end == n:
            break
        start = end-over_lap

    return chunks

In [9]:
chunks = chunk_text(clean_text)
print(len(chunks))

30


In [10]:
chunks[29]

'信，保留对您所提供个⼈资料进⾏审查的权利；如发现任何虚假或不实信息，公\n司将视情节严重程度，随时终⽌您的试⽤或解除劳动合同，并由您独⽴承担由此产⽣的全部后果和法\n律责任。\n我们的各项规章制度在持续完善中，欢迎您随时将合理化建议反馈⾄⼈⼒资源部；如对任何政策或流\n程存有疑问，也可向⼈事⾏政部咨询。良好的企业⽂化源于每位成员的⾃觉遵守与⽇常践⾏⸺希望您\n以饱满的热情、积极的创新精神，与我们共同营造开放、务实、和谐的⼯作氛围。只要您在 AFY 这个\n舞台上不断挑战⾃我、持续进步，我们定将为您提供更⼴阔的发展空间与成⻓平台。\n本制度适⽤于杭州翱孚尤教育科技有限公司及其所有关联公司（包括未来新成⽴的关联实体）。在\n此，您承诺已完整阅读本《员⼯⼿册》，并就相关内容向⼈⼒资源部进⾏过咨询，充分理解并认同其\n中所有规定，愿意⾃觉遵守，携⼿共创美好未来。\n  \n员⼯签字：  \n⽇  期： \n⼈⼒资源部'

## Embeddings

In [11]:
from sentence_transformers import SentenceTransformer

embedding = SentenceTransformer("BAAI/bge-base-zh-v1.5")

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /BAAI/bge-base-zh-v1.5/resolve/main/modules.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x1619f5be0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: d0ef28af-0946-4f73-9095-00ef2a7912c2)')' thrown while requesting HEAD https://huggingface.co/BAAI/bge-base-zh-v1.5/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /BAAI/bge-base-zh-v1.5/resolve/main/modules.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x1619eae90>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 865b7d2e-e36d-4737-9265-346fd9e11890)')' thrown while requesting HEAD https://huggingface.co/BAAI/bge-base-zh-v1.5/resolve/main/./modules.json
Retrying in 2s [Retry 2/5].
'(MaxRetry

In [12]:

def embed(texts):
    vecs = embedding.encode(texts, convert_to_numpy=True, normalize_embeddings=True)
    return vecs
test_vec = embed(["你好"])
test_vec.shape


(1, 768)

In [13]:
chunks_embeddings = embed(chunks)
print(chunks_embeddings.shape)
print(chunks_embeddings.dtype)

(30, 768)
float32


## VectorDB and Retriever

### 第一次失败

In [14]:
import faiss
import numpy as np

index = faiss.IndexFlatL2(chunks_embeddings.shape[1])
index.add(chunks_embeddings)

In [15]:
#test on small query
query_embed = embed(["请问入职审核多久"]).astype("float32")
dist, idx = index.search(query_embed, k=3)
print(dist, idx)

[[0.76370126 0.7887718  0.8047117 ]] [[21 22 29]]


In [16]:
for i, id in enumerate(idx[0]):
    print("="*40)
    print("Rank:", i+1)
    print("Distance:", dist[0][i])
    print("Chunk index:", id)
    print(chunks[id])


Rank: 1
Distance: 0.76370126
Chunk index: 21
资源相关
5. 处罚办法 
违反以上红线⾏为之⼀，视为严重违反规章制度。视情节严重者，公司将有权解除劳动关系并不需⽀
付任何经济补偿。若造成公司的经济损失，追究其经济赔偿；若违反法律法规的将移交公安机关处
理。（最终解释权归公司所有）
  
三、新员⼯试⽤期管理
1. 试⽤期期限 
1.1 我司正式员⼯签订 3 年（含）及以上固定期限劳动合同的，试⽤期为 6 个⽉；签订 1 年（含）及以
上不满 3 年固定期限劳动合同的，试⽤期为 2 个⽉；临时⽤⼯⼀般签订三个⽉（含）及以上不满 1 年
固定期限劳动合同的 ，试⽤期为 1 个⽉；低于3个⽉的⼀般设 1 周考察期。如试⽤期表现优异，可视实
际情况可安排提前转正考核。
1.2 试⽤期如果请事病假或产假的，相应的假期不计⼊试⽤期间，试⽤期要做相应的顺延。 
2. 试⽤期考核 
2.1 试⽤期考核⽬标确定 
新员⼯⼊职⼀周内，由⽤⼈部⻔负责协助该
Rank: 2
Distance: 0.7887718
Chunk index: 22
况可安排提前转正考核。
1.2 试⽤期如果请事病假或产假的，相应的假期不计⼊试⽤期间，试⽤期要做相应的顺延。 
2. 试⽤期考核 
2.1 试⽤期考核⽬标确定 
新员⼯⼊职⼀周内，由⽤⼈部⻔负责协助该员⼯制定第⼀个⽉的试⽤期考核⽬标，考核⽬标经直线上
级审批通过后，开始考核。每⽉底评估当⽉考核⽬标，并拟定次⽉考核⽬标。15⽇前⼊职，当⽉考
核；15⽇及以后⼊职，和次⽉合并考核。考核周期按⾃然⽉。⼈⼒资源部应协助部⻔完成新⼈培育及
考核⼯作。
2.2 试⽤期考核内容 
公司在试⽤期内将对新员⼯进⾏以下内容的考核 ，考核形式包括⼝试和业务过程考察等⽅式。 
2.2.1 考核项及权重 
考核项 
前台业务部⻔  其他部⻔ 
参考内容  权重  参考内容  权重 
能⼒评估 
（⼯作业绩表
现） 
签单量 
销售能⼒ 
70%  项⽬完成情况（如业绩是否完成等)  
任务处理时效性 
70% 
学
Rank: 3
Distance: 0.8047117
Chunk index: 29
信，保留对您所提供个⼈资料进⾏审查的权利；如发现任何虚假或不实信息，公
司将视情节严重程度，随时终⽌您的试⽤或解除劳动合同，并由您

### 第二次尝试

重新chunk！

In [17]:
chunks = chunk_text(clean_text, chunk_size = 400, over_lap = 100)
print(len(chunks))

30


In [18]:
chunks_embeddings = embed(chunks)

In [19]:
def retrieve(query, top_k = 3):
    query_vec = embed([query]).astype("float32")
    dist, idx = index.search(query_vec, top_k)
    results = []
    for rank, (d, i) in enumerate(zip(dist[0], idx[0])):
        results.append({
            "rank": rank+1,
            "chunk_id": int(i),
            "distance": float(d),
            "text": chunks[i]
        })
    return results

In [20]:
query = "年假怎么休？"
results = retrieve(query)
results

[{'rank': 1,
  'chunk_id': 11,
  'distance': 0.9687948226928711,
  'text': '当年的年休假： \n• 员⼯⼯作满1年不满10年，请病假累计1个⽉及以上的； \n• 累计⼯作满10年不满20年的职⼯，请病假累计1.5个⽉及以上的； \n• 累计⼯作满20年以上的职⼯，请病假累计2个⽉及以上的。 \n（4）公司根据实际经营情况统筹安排员⼯年休假，可在1个年度内集中安排，也可以分段安排，⼀般\n不跨年度安排。但因部分岗位特点确有必要跨年度安排职⼯年休假的，可以跨1个年度安排。 \n（5）员⼯可⾃⾏申请年休假，但公司确因⼯作需要不能安排职⼯休年休假的，可以拒绝员⼯提交的年\n休假申请，后续统⼀统筹安排。\n⼯龄（n/年）  年休假天数 \n1≤n＜10  5天 \n10≤n＜20  10天 \n20≤n  15天 \n12.2 事假 \n• 事假是指因私事或其他个⼈原因需请的假；\n• 事假⼀天扣除⽇⼯资（含补贴等部分）的100%； \n• 当⽉因请事假致员⼯当⽉发放⼯资不⾜以扣除个⼈应缴纳的公积⾦及保'},
 {'rank': 2,
  'chunk_id': 10,
  'distance': 0.9859317541122437,
  'text': '假期，需提前2天申请，并有⼯作交接，经上级领导审批后⽅可离岗。 \n• 假期开始前必须确认完成当前正在进⾏的⼯作，并做好放假前的交接后，⽅可离岗，因交接不周给\n公司带来损失的必须承担相 应责任。 \n• ⽆请假审批⼿续且未到岗，或请假审批上级领导未通过⽽未到岗， ⼀律以旷⼯处理。 \n注：所有请假、出差等的最⼩单位为半天。\n12. 假期类别 \n12.1 年休假 \n（1）根据国家相关规定，结合公司实际业务情况，员⼯享受带薪年休假。 \n（2）员⼯累计⼯作已满1年不满10年的，年休假5天；已满10年不满20年的，年休假10天；已满20年\n的，年休假15天。（详⻅下表） \n（3）员⼯有下列情形之⼀的，不享受当年的年休假： \n• 员⼯⼯作满1年不满10年，请病假累计1个⽉及以上的； \n• 累计⼯作满10年不满20年的职⼯，请病假累计1.5个⽉及以上的； \n• 累计⼯作满20年以上的职⼯，请病假累计2个⽉及'},
 {'rank': 3,

## Build a re-prompt Agent

有的时候用户的问题 太短，没办法检索。 需要 加一个 小的agent来帮忙

In [31]:
def reprompt_agent(query):
    """
    Transfer the original query to a more complete question for further retrieve
    """
    system_msg = """
    你是中文re-prompt agent，帮助用户把模糊或短的问题写完整，用于后续RAG检索
    """
    user_msg = f"""
    你会收到用户关于企业规章&人力资源管理相关的query。请先理解用户query，并站在员工立场把query补充成完整，
    明确的问题，用于后续RAG检索。 仅输出改善后的中文query。
    用户的query:{query}
    """
    response = client.chat.completions.create(
        messages = [
    {"role":"system", "content":system_msg},
    {"role":"user", "content": user_msg},
        ],
        model = "gpt-5-nano",
    )

    new_query = response.choices[0].message.content
    return new_query
    
        

In [32]:
client = OpenAI(
    base_url = 'https://api.zhizengzeng.com/v1',
    api_key = api_key,
)

reprompt_agent("年假怎么休？")

'请以员工视角，详细解答贵公司关于年假的完整规定与流程：包括年假天数的计算与适用对象（含试用期/在职时长、是否可结转及上限）、年假申请的流程与时限（提前多久、提交方式/系统、需要的材料、审批人及耗时、请假单模板）、休假期间工资与福利的影响、与病假/事假/婚假等假别的关系及冲突处理、是否存在休假高峰期限制、以及查询年假余额的途径。'

ok！ 大成功！！！！！！！

大成功！ ok，继续！

## Rag 

In [37]:
def LLM_using_RAG(query,model):
    #first, we reprompt the query
    new_query = reprompt_agent(query)

    #print("STEP 1: Reprompt =======")
    #print(new_query)
    #next, retrieve
    retrieved_results = retrieve(new_query)
    #print("STEP 2: Retrive Relevant Info ======")
    
    #print(retrieved_results)
    system_prompt = f"""
    你是企业的HR助手，善于回答员工关于企业相关问题
    """

    user_prompt = f"""
    你会收到用户的query和一份参考资料。
    要求：
    1. 你有你自己的常识，你可以选择用或者不用参考资料
    2. 请以HR的口吻来做回答
    3. 如果用了参考资料，请在回答中包含参考资料的原文和reference

    用户的query:{query}
    参考资料:{retrieved_results}
    """
    response = client.chat.completions.create(
        messages = [
            {"role":"system", "content":system_prompt},
            {"role":"user", "content": user_prompt},
        ],
        model = model,
    )

    #print("STEP 3: LLM use results to generate Answer ====")
    return response.choices[0].message.content
    

In [38]:
query = "年假怎么休？"
response = LLM_using_RAG(query, model = "gpt-5-nano")
response

STEP 1: Reprompt =======
请以员工视角，获取本公司年假制度的完整操作指引：年假天数的计算与累积规则、是否可跨年度使用、年假上限与是否可转结/补休、请假提前通知时限、申请流程与审批人（线上入口、需要提交的材料、是否需主管批准）、年假工资结算规则（按工作日还是实际请假天数、是否发放全额、扣税等）、与法定节假日、周末的叠加或冲突规则、排班与在岗要求、离职前的年假清算及未休年假的处理、以及常见问题解答或申请模板入口。
STEP 2: Retrive Relevant Info ======
[{'rank': 1, 'chunk_id': 10, 'distance': 0.2691398859024048, 'text': '假期，需提前2天申请，并有⼯作交接，经上级领导审批后⽅可离岗。 \n• 假期开始前必须确认完成当前正在进⾏的⼯作，并做好放假前的交接后，⽅可离岗，因交接不周给\n公司带来损失的必须承担相 应责任。 \n• ⽆请假审批⼿续且未到岗，或请假审批上级领导未通过⽽未到岗， ⼀律以旷⼯处理。 \n注：所有请假、出差等的最⼩单位为半天。\n12. 假期类别 \n12.1 年休假 \n（1）根据国家相关规定，结合公司实际业务情况，员⼯享受带薪年休假。 \n（2）员⼯累计⼯作已满1年不满10年的，年休假5天；已满10年不满20年的，年休假10天；已满20年\n的，年休假15天。（详⻅下表） \n（3）员⼯有下列情形之⼀的，不享受当年的年休假： \n• 员⼯⼯作满1年不满10年，请病假累计1个⽉及以上的； \n• 累计⼯作满10年不满20年的职⼯，请病假累计1.5个⽉及以上的； \n• 累计⼯作满20年以上的职⼯，请病假累计2个⽉及'}, {'rank': 2, 'chunk_id': 11, 'distance': 0.33988162875175476, 'text': '当年的年休假： \n• 员⼯⼯作满1年不满10年，请病假累计1个⽉及以上的； \n• 累计⼯作满10年不满20年的职⼯，请病假累计1.5个⽉及以上的； \n• 累计⼯作满20年以上的职⼯，请病假累计2个⽉及以上的。 \n（4）公司根据实际经营情况统筹安排员⼯年休假，可在1个年度内集中安排，也可以分段安排，⼀般\n不跨年度安排。但因部分岗位特点确有必要跨年度安排职⼯年休假的

'您好，关于“年假怎么休”的要点，整理如下，供您快速了解和操作。\n\n一、年假天数与享有条件\n- 工龄1年（含）至不足10年：年休假5天\n- 10年至不足20年：年休假10天\n- 20年以上：年休假15天\n- 不享受当年的年休假的情形（需注意）：\n  • 员工工作满1年不满10年，请病假累计1个月及以上的；\n  • 累计工作满10年不满20年的员工，请病假累计1.5个月及以上的；\n  • 累计工作满20年以上的员工，请病假累计2个月及以上的。\n\n二、请假/年休假需要的申请与交接流程\n- 请假（含年休假）需提前申请：假期，需提前2天申请，并有工作交接，经上级领导审批后方可离岗。\n- 休假前要确认完成当前正在进行的工作，并做好放假前的交接后，方可离岗，因交接不周给公司带来损失的必须承担相应责任。\n- 注：所有请假、出差等的最小单位为半天。\n\n三、年休假安排的时间分配\n- 公司根据实际经营情况统筹安排员工年休假，可在1个年度内集中安排，也可以分段安排，一般不跨年度安排。但因部分岗位特点确有必要跨年度安排职工年休假的，可以跨1个年度安排。\n- 员工可自行申请年休假，但公司因工作需要不能安排职工休年休假的，可以拒绝员工提交的年休假申请，后续统筹安排。\n\n四、实际操作小提示\n- 如果您想要我帮您初步核算您本年度的年休假天数，请提供您的工龄，以及近年的病假累计情况（若有具体数字也可直接告诉我）。\n- 最小单位为半天，请在申请时以半天为单位进行申请。\n\n原文摘录（选取自您提供的参考资料）：\n- “假期，需提前2天申请，并有⼯作交接，经上级领导审批后⽅可离岗。”\n- “假期开始前必须确认完成当前正在进⾏的⼯作，并做好放假前的交接后，⽅可离岗，因交接不周给 公司带来损失的必须承担相 应责任。”\n- “注：所有请假、出差等的最⼩单位为半天。”\n- “12.1 年休假 …（2）员工累计⼯作已满1年不满10年的，年休假5天；已满10年不满20年的，年休假10天；已满20年的，年休假15天。”\n- “11.1 请假规定 … ⼀律需提前2天申请，并有⼯作交接，经上级领导审批后⽅可离岗。 … 休假前必须完成当前工作的交接后才能离岗。”\n- “12.2 事假 … 事假⼀天扣除⽇⼯资 …”\n\n参考资料来源（reference）\n- 参考资

In [39]:
print(response)

您好，关于“年假怎么休”的要点，整理如下，供您快速了解和操作。

一、年假天数与享有条件
- 工龄1年（含）至不足10年：年休假5天
- 10年至不足20年：年休假10天
- 20年以上：年休假15天
- 不享受当年的年休假的情形（需注意）：
  • 员工工作满1年不满10年，请病假累计1个月及以上的；
  • 累计工作满10年不满20年的员工，请病假累计1.5个月及以上的；
  • 累计工作满20年以上的员工，请病假累计2个月及以上的。

二、请假/年休假需要的申请与交接流程
- 请假（含年休假）需提前申请：假期，需提前2天申请，并有工作交接，经上级领导审批后方可离岗。
- 休假前要确认完成当前正在进行的工作，并做好放假前的交接后，方可离岗，因交接不周给公司带来损失的必须承担相应责任。
- 注：所有请假、出差等的最小单位为半天。

三、年休假安排的时间分配
- 公司根据实际经营情况统筹安排员工年休假，可在1个年度内集中安排，也可以分段安排，一般不跨年度安排。但因部分岗位特点确有必要跨年度安排职工年休假的，可以跨1个年度安排。
- 员工可自行申请年休假，但公司因工作需要不能安排职工休年休假的，可以拒绝员工提交的年休假申请，后续统筹安排。

四、实际操作小提示
- 如果您想要我帮您初步核算您本年度的年休假天数，请提供您的工龄，以及近年的病假累计情况（若有具体数字也可直接告诉我）。
- 最小单位为半天，请在申请时以半天为单位进行申请。

原文摘录（选取自您提供的参考资料）：
- “假期，需提前2天申请，并有⼯作交接，经上级领导审批后⽅可离岗。”
- “假期开始前必须确认完成当前正在进⾏的⼯作，并做好放假前的交接后，⽅可离岗，因交接不周给 公司带来损失的必须承担相 应责任。”
- “注：所有请假、出差等的最⼩单位为半天。”
- “12.1 年休假 …（2）员工累计⼯作已满1年不满10年的，年休假5天；已满10年不满20年的，年休假10天；已满20年的，年休假15天。”
- “11.1 请假规定 … ⼀律需提前2天申请，并有⼯作交接，经上级领导审批后⽅可离岗。 … 休假前必须完成当前工作的交接后才能离岗。”
- “12.2 事假 … 事假⼀天扣除⽇⼯资 …”

参考资料来源（reference）
- 参考资料原文摘录来自您提供的内容中的相关段落，包含 chunk_id 10